# Instance Segmentation with Detectron2 and Remo

In this tutorial, we do transfer learning on a MaskRCNN model from Detectron2. 
We use Remo to facilitate exporing, accessing and managing the dataset.




In particular, we will:

* Browse through our images and annotations
* Quickly visualize the main properties of the dataset and annotations
* Create a train, test, valid split without moving data around, using Remo image tags.
* Fine tune a pre-trained MaskRCNN model from Detectron2 and do some inference
* Visually compare Mask predictions with the ground truth



**Along the way, we will see how browsing images, annotations and predictions helps to gather insights on the dataset and on the model.**

Before proceeding, we need to install the required dependencies. 

This can be done by executing the next cell. Once complete, **restart your runtime** to ensure that the installed packages can be detected.



In [16]:
#!pip install imantics
#!pip install git+https://github.com/facebookresearch/fvcore.git
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
#!pip install -e detectron2_repo

Cloning into 'detectron2_repo'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 7887 (delta 8), reused 22 (delta 8), pack-reused 7856
Receiving objects: 100% (7887/7887), 3.47 MiB | 3.04 MiB/s, done.
Resolving deltas: 100% (5640/5640), done.


In [ ]:
from google.colab import drive
GDRIVE_ROOT = "/gdrive"
drive.mount(GDRIVE_ROOT)

In [ ]:
!pip install "/gdrive/My Drive/remo-0.5.2.1-py3-none-any.whl"
!python -m remo_app init --colab

Let us then import the required packages.

In [2]:
import remo
remo.set_viewer('jupyter')

import numpy as np
import os
from PIL import Image
import glob
import random
random.seed(4)

from imantics import Polygons, Mask

import torch, torchvision

# Detectron 2 files
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.data.datasets import register_coco_instances

### Adding Data to Remo

* The Dataset used is a subset of the <a href="https://cocodataset.org/#home">MS COCO Dataset</a>.
* The directory structure of the dataset is:
```
├── instance_segmentation_dataset
    ├── images
        ├── image_1.jpg
        ├── image_2.jpg
        ├── ...
    ├── annotations
        ├── dataset_annotations.json
```


In [3]:
if not os.path.exists('instance_segmentation_dataset.zip'):
  !wget https://s-3.s3-eu-west-1.amazonaws.com/instance_segmentation_dataset.zip
  !unzip instance_segmentation_dataset.zip
else:
  print('Files already downloaded')

--2020-10-28 13:30:04--  https://s-3.s3-eu-west-1.amazonaws.com/instance_segmentation_dataset.zip
Resolving s-3.s3-eu-west-1.amazonaws.com (s-3.s3-eu-west-1.amazonaws.com)... 52.218.100.8
Connecting to s-3.s3-eu-west-1.amazonaws.com (s-3.s3-eu-west-1.amazonaws.com)|52.218.100.8|:443...connected.
HTTP request sent, awaiting response...200 OK
Length: 1720189 (1.6M) [application/zip]
Saving to: ‘instance_segmentation_dataset.zip’

instance_segmentati 100%[===================>]   1.64M  1.06MB/s    in 1.5s    

2020-10-28 13:30:06 (1.06 MB/s) - ‘instance_segmentation_dataset.zip’ saved [1720189/1720189]

Archive:  instance_segmentation_dataset.zip
   creating: instance_segmentation_dataset/
   creating: instance_segmentation_dataset/annotations/
  inflating: instance_segmentation_dataset/annotations/coco_subset.json  
   creating: instance_segmentation_dataset/images/
  inflating: instance_segmentation_dataset/images/000000520301.jpg  
  inflating: instance_segmentation_dataset/images/0000

In [5]:
path_to_annotations = 'instance_segmentation_dataset/annotations/'
path_to_images = 'instance_segmentation_dataset/images/'

## Train / test split
In Remo, we can use tags to organise our images. Among other things, this allows us to generate train / test splits without the need to move image files around.

To do this, we just need to pass a dictionary (mapping tags to the relevant images paths) to the function ```remo.generate_image_tags()```.

In [6]:
im_list = [i for i in glob.glob(path_to_images + '/**/*.jpg', recursive=True)]
im_list = random.sample(im_list, len(im_list))

train_idx = round(len(im_list) * 0.8)
test_idx  = train_idx + round(len(im_list) * 0.2)

tags_dict =  {'train' : im_list[0:train_idx], 
              'test' : im_list[train_idx:test_idx]}

train_test_split_file_path = os.path.join(path_to_annotations, 'images_tags.csv') 
remo.generate_image_tags(tags_dictionary  = tags_dict, 
                         output_file_path = train_test_split_file_path, 
                         append_path = False)

'instance_segmentation_dataset/annotations/images_tags.csv'

### Create a dataset

To create a dataset we can use ```remo.create_dataset()```, specifying the path to data and annotations.

For a complete list of formats supported, you can <a href="https://remo.ai/docs/annotation-formats/"> refer to the docs</a>.


In [7]:
instance_segmentation_dataset = remo.create_dataset(name = 'coco_subset', local_files = [path_to_annotations, path_to_images], annotation_task='Instance Segmentation')

Acquiring data - completed                                                                           
Processing annotation files: 1 of 2 filesProcessing data - completed                                                                          
Data upload completed with some errors:
000000289343.jpg: No valid url
000000061471.jpg: No valid url
000000472375.jpg: No valid url
000000520301.jpg: No valid url
000000064359.jpg: No valid url
000000486046.jpg: No valid url
000000349302.jpg: No valid url
000000509735.jpg: No valid url
000000250758.jpg: No valid url
000000562121.jpg: No valid url


**Visualizing the dataset**

To view and explore images and labels, we can use Remo directly from the notebook. We just need to call ```dataset.view()```.

In [6]:
instance_segmentation_dataset.view()

Open http://localhost:8123/datasets/2


**Dataset Statistics**

Using Remo, we can quickly visualize some key Dataset properties that can help us with our modelling, without needing to write extra boilerplate code.

This can be done either from code, or using the visual interface.

In [8]:
instance_segmentation_dataset.get_annotation_statistics()

[{'AnnotationSet ID': 3,
  'AnnotationSet name': 'Instance segmentation',
  'n_images': 10,
  'n_classes': 10,
  'n_objects': 35,
  'top_3_classes': [{'name': 'Zebra', 'count': 13},
   {'name': 'Giraffe', 'count': 9},
   {'name': 'Dog', 'count': 4}],
  'creation_date': None,
  'last_modified_date': '2020-10-28T13:31:29.901116Z'}]

In [10]:
instance_segmentation_dataset.view_annotation_stats()

Open http://localhost:8123/annotation-detail/3/insights


**Exporting the dataset**

To export annotations according to the train, test split in a format accepted by the model, we use the ```dataset.export_annotations_to_file()``` method, and filter by the desired tag.

For a complete list of formats supported, you can <a href="https://remo.ai/docs/annotation-formats/"> refer to the docs</a>.

In [6]:
path_to_train = path_to_annotations + "instance_segmentation_train.json"
path_to_test = path_to_annotations + "instance_segmentation_test.json"

In [12]:
instance_segmentation_dataset.export_annotations_to_file(path_to_train, annotation_format='coco', filter_by_tags=['train'], export_tags=False, append_path=False)
instance_segmentation_dataset.export_annotations_to_file(path_to_test, annotation_format='coco', filter_by_tags=['test'], export_tags=False, append_path=False)

# Detectron2

Here we will start working with the ```Detectron2``` framework written in PyTorch.

## Feeding Data into Detectron2

To use Detectron2, you are required to register your dataset.

The ```register_coco_instances``` method takes in the following parameters:

* **path_to_annotations:** Path to annotation files. Format: COCO JSON.
* **path_to_images:** Path to the folder containing the images.

This then allows to store the metadata for future operations.

In [9]:
register_coco_instances('instance_segmentation_train', {}, path_to_train, path_to_images)
register_coco_instances('instance_segmentation_test', {}, path_to_test, path_to_images)

train_metadata = MetadataCatalog.get('instance_segmentation_train')

## Training the Model

For the sake of the tutorial, our ```Mask RCNN``` architecture will have a ```ResNet-50 Backbone```, pre-trained on on COCO train2017. This can be loaded directly from Detectron2.

To train the model, we specify the following details:

- **model_yaml_path:** Configuration file for the Mask RCNN model.
- **model_weights_path**: Symbolic link to the desired Mask RCNN architecture.

The parameters can be tweaked by overriding the correspodning variable in the ```cfg```.

In [15]:
ls

colab_instance_segmentation_tutorial.ipynb  instance_segmentation_dataset.zip
instance_segmentation_dataset/


In [10]:
model_yaml_path = './detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml'
model_weights_path = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl'

cfg = get_cfg()
cfg.merge_from_file(model_yaml_path)
cfg.DATASETS.TRAIN = ('instance_segmentation_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_weights_path # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MAX_ITER = 150    # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10

**Instantiating the Trainer**

We instatiate the trainer with the required configuration, and finally kick-off the training.

In [11]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[10/28 13:44:14 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

## Visualizing Predictions

Using Remo, we can easily browse our predictions and compare them with the ground-truth.

We will do this by uploading the model predictions to a new ```AnnotationSet```, which we call `model_predictions`

To visualise the labels as strings rather than IDs, we can use a dictionary mapping the two of them.

In [12]:
mapping = {k: v for k, v in enumerate(train_metadata.thing_classes)}

In [14]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set the testing threshold for this model
cfg.DATASETS.TEST = ("instance_segmentation_test", )
predictor = DefaultPredictor(cfg)
test_dataset_dicts = DatasetCatalog.get("instance_segmentation_test")

WARNING [10/28 13:50:36 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[10/28 13:50:36 d2.data.datasets.coco]: Loaded 2 images in COCO format from instance_segmentation_dataset/annotations/instance_segmentation_test.json


In [15]:
for d in test_dataset_dicts:    
    im = np.array(Image.open(d["file_name"]))
    outputs = predictor(im)
    pred_classes = outputs['instances'].get('pred_classes').cpu().numpy()
    masks = outputs['instances'].get("pred_masks").cpu().permute(1, 2, 0).numpy()
    image_name = d['file_name']
    annotations = []
    
    if masks.shape[2] != 0:
        for i in range(masks.shape[2]):
            polygons = Mask(masks[:, :, i]).polygons()
            annotation = remo.Annotation()
            annotation.img_filename = image_name
            annotation.classes = mapping[pred_classes[i]]
            annotation.segment = polygons.segmentation[0]
            annotations.append(annotation)
    else:
        polygons = Mask(masks[:, :, 0]).polygons()
        annotation = remo.Annotation()
        annotation.img_filename = image_name
        annotation.classes = mapping[pred_classes[0]]
        annotation.segment = polygons.segmentation[0]
        annotations.append(annotation)

In [18]:
instance_segmentation_dataset = remo.get_dataset(3)

In [19]:
model_predictions = instance_segmentation_dataset.create_annotation_set(annotation_task = 'Instance Segmentation', name = 'model_predictions')

instance_segmentation_dataset.add_annotations(annotations, annotation_set_id=model_predictions.id)

Progress 100% - 1/1 - elapsed 0:00:00.001000 - speed: 1000.00 img / s, ETA: 0:00:00
Acquiring data - completed                                                                           
Processing data - completed                                                                          
Data upload completed


In [20]:
instance_segmentation_dataset.view()

Open http://localhost:8123/datasets/3


By visualizing the predicted masks against the ground truth, we can go past summary performance metrics, and visually inspect model biases and iterate to improve it.